# Introduction:

This Jupyter Notebook explores a dataset containing information about Airbnb listings in Cape Town. The dataset comprises various attributes such as listing ID, host details, geographical coordinates, room type, pricing, availability, and review-related metrics. With over 22,000 entries and 18 columns, the dataset offers a comprehensive overview of the Airbnb market in Cape Town.

In this project, we aim to analyze and understand the dynamics of the Airbnb market in Cape Town. By leveraging the power of Python and the Pandas library, we will delve into the dataset to uncover insights, trends, and patterns that can provide valuable information for both hosts and guests alike.

Our analysis will cover a wide range of aspects, including pricing trends, popularity of room types, host behaviors, geographical distribution of listings, and more. Through exploratory data analysis (EDA), visualization, and statistical techniques, we will gain deeper insights into the Airbnb ecosystem in the area.

Ultimately, this project seeks to offer actionable insights and recommendations for various stakeholders, including Airbnb hosts, prospective guests, and policymakers, to make informed decisions and enhance their experiences within the Airbnb community.

In [271]:
import numpy as np     
import pandas as pd      
import matplotlib.pyplot as plt   
import seaborn as sns 
import re

# Statistical analysis
import scipy.stats as stats

# Jupyter Notebook magic commands for inline plotting
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [272]:
df = pd.read_csv('data/raw/listings.csv')

### Understanding the data

In [273]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,3191,Home in Southern Suburbs · ★4.81 · 1 bedroom ·...,3754,Brigitte,NaN,Ward 57,-33.947620,18.475990,Entire home/apt,650.0,3,75,2023-12-23,0.58,1,306,6,NaN
1,15007,Home in Cape Town · ★4.82 · 3 bedrooms · 4 bed...,59072,Dirk,NaN,Ward 23,-33.800010,18.460630,Entire home/apt,4029.0,2,45,2023-12-03,0.37,3,294,6,NaN
2,15068,Rental unit in Cape Town · 3 bedrooms · 5 beds...,59318,Linda,NaN,Ward 23,-33.788260,18.459400,Entire home/apt,2000.0,4,0,NaN,NaN,4,354,0,NaN
3,15077,Rental unit in Tableview - Sunset Beach · ★5.0...,59342,Georg,NaN,Ward 4,-33.858356,18.490376,Private room,2377.0,2,7,2022-06-16,0.05,6,83,0,NaN
4,15199,Rental unit in Cape Town · 1 bedroom · 1 bed ·...,59694,Alexa,NaN,Ward 115,-33.911150,18.412350,Entire home/apt,2500.0,14,2,2016-04-15,0.02,1,365,0,NaN


In [274]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22966 entries, 0 to 22965
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              22966 non-null  int64  
 1   name                            22966 non-null  object 
 2   host_id                         22966 non-null  int64  
 3   host_name                       22962 non-null  object 
 4   neighbourhood_group             0 non-null      float64
 5   neighbourhood                   22966 non-null  object 
 6   latitude                        22966 non-null  float64
 7   longitude                       22966 non-null  float64
 8   room_type                       22966 non-null  object 
 9   price                           21574 non-null  float64
 10  minimum_nights                  22966 non-null  int64  
 11  number_of_reviews               22966 non-null  int64  
 12  last_review                     

In [275]:
df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'number_of_reviews_ltm', 'license'],
      dtype='object')

## 1. Data Preprocessing 

### 1.1 Preprocessing: Name Column

The first part of the preprocessing will involve the `name` column. We will split the data into five columns, namely:

- `property type`
- `rating`
- `bedrooms`
- `beds`
- `baths`


In [276]:
# create property info dataframe

df_prop_inf = pd.DataFrame()

In [277]:
# split name column into five columns

df_prop_inf[['property_type', 'rating', 'no._of_bedrooms', 'no._of_beds', 'no._of_baths']] = df['name'].str.split(' · ', expand=True)
df_prop_inf.head()

,property_type,rating,no._of_bedrooms,no._of_beds,no._of_baths
0,Home in Southern Suburbs,★4.81,1 bedroom,1 bed,1 bath
1,Home in Cape Town,★4.82,3 bedrooms,4 beds,3 baths
2,Rental unit in Cape Town,3 bedrooms,5 beds,2 baths,None
3,Rental unit in Tableview - Sunset Beach,★5.0,1 bedroom,2 beds,1.5 baths
4,Rental unit in Cape Town,1 bedroom,1 bed,1 bath,None


### Work on Property Type column:

In [278]:
# check if property type column only contains property type data 

percentage_with_in = (df_prop_inf['property_type'].str.contains('in').sum() / len(df_prop_inf)) * 100
print("Percentage of rows with 'in' in the 'property type' column:", percentage_with_in)

Percentage of rows with 'in' in the 'property type' column: 100.0


In [279]:
# isolate property type column for further cleaning by splitting the property type and area

df_prop_inf_clean = df_prop_inf['property_type'].str.split(' in ', expand=True)
df_prop_inf_clean.head(2)

,0,1,2
0,Home,Southern Suburbs,None
1,Home,Cape Town,None


In [280]:
# check which row does not have null value because the 3rd column should not exist
df_prop_inf_clean[df_prop_inf_clean.notna().all(axis=1)]

,0,1,2
1920,Rental unit,Simon's Town,Cape Town


In [281]:
# change all 3rd column values to None

df_prop_inf_clean.loc[df_prop_inf_clean[2].notna(), 2] = None
print(df_prop_inf_clean[2].value_counts())

Series([], Name: count, dtype: int64)


In [282]:
# drop third column

df_prop_inf_clean = df_prop_inf_clean.drop(columns=[2])

In [283]:
# change 2nd column values to 'Cape Town'

df_prop_inf_clean[1] = 'Cape Town'

In [284]:
# change column names

df_prop_inf_clean.columns = ['property_type', 'city']
df_prop_inf_clean.head(2)

,property_type,city
0,Home,Cape Town
1,Home,Cape Town


In [285]:
# remove the first column from df

df_prop_inf = df_prop_inf.iloc[:, 1:]

In [286]:
# add clean property type and city columns to property info df

df_prop_inf['property_type'] = df_prop_inf_clean['property_type']
df_prop_inf['city'] = df_prop_inf_clean['city']

df_prop_inf.head(2)

,rating,no._of_bedrooms,no._of_beds,no._of_baths,property_type,city
0,★4.81,1 bedroom,1 bed,1 bath,Home,Cape Town
1,★4.82,3 bedrooms,4 beds,3 baths,Home,Cape Town


In [287]:
df[['property_type', 'city']] = df_prop_inf[['property_type', 'city']]

In [288]:
df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'number_of_reviews_ltm', 'license', 'property_type',
       'city'],
      dtype='object')


The `property type` and `city` columns have been cleaned and added to the original DataFrame.

### Work on ratings column

In [289]:
# create ratings dataframe

df_ratings = pd.DataFrame()

In [290]:
# populate with ratings column from original dataframe

df_ratings = df_prop_inf['rating'].copy()
df_ratings.head(3)

0         ★4.81
1         ★4.82
2    3 bedrooms
Name: rating, dtype: object

In [291]:
#if the value does not contain a star, change it to No Rating

df_ratings[df_ratings.str.contains('★') == False ] = 'No rating'
value_counts = df_ratings.value_counts()
no_rating_count = value_counts.get('No rating', 0)
print("Number of 'No rating' values:", no_rating_count)

Number of 'No rating' values: 7288


In [292]:
# if value contains the string New, change it to No Rating

df_ratings[df_ratings.str.contains('New') == True ] = 'No rating'
value_counts = df_ratings.value_counts()
no_rating_count = value_counts.get('No rating', 0)
print("Number of 'No rating' values:", no_rating_count)

Number of 'No rating' values: 9339


In [293]:
# remove the star

df_ratings.loc[df_ratings.str.contains('.')] = df_ratings.str.replace('★', '')
df_ratings.head()

0         4.81
1         4.82
2    No rating
3          5.0
4    No rating
Name: rating, dtype: object

In [294]:
# replace No rating with NaN

df_ratings.replace('No rating', np.nan, inplace=True)
df_ratings.head()

0    4.81
1    4.82
2     NaN
3     5.0
4     NaN
Name: rating, dtype: object

In [295]:
# change series to float values

df_ratings = df_ratings.astype(float)
df_ratings.info()

<class 'pandas.core.series.Series'>
RangeIndex: 22966 entries, 0 to 22965
Series name: rating
Non-Null Count  Dtype  
--------------  -----  
13627 non-null  float64
dtypes: float64(1)
memory usage: 179.6 KB


In [296]:
# create rating column

df['rating'] = df_ratings


The `rating` column has been cleaned and added to the original DataFrame.

### Work on the bedroom column

In [297]:
# create bedrooms dataframe

df_bedrooms = pd.DataFrame()

In [298]:
# create copy of dataframe

df_bedrooms = df_prop_inf[['rating', 'no._of_bedrooms']].copy()
df_bedrooms.head()

,rating,no._of_bedrooms
0,★4.81,1 bedroom
1,★4.82,3 bedrooms
2,3 bedrooms,5 beds
3,★5.0,1 bedroom
4,1 bedroom,1 bed


In [299]:
# define a custom function to assign values to the 'bedrooms' column based on the 'rating' column

def assign_bedrooms(row):
    if 'bedroom' in row['rating']:
        return row['rating']
    else:
        return row['no._of_bedrooms']

# apply the custom function to each row of the DataFrame

df_bedrooms['no._of_bedrooms'] = df_bedrooms.apply(assign_bedrooms, axis=1)

In [300]:
# extract integers from the 'bedrooms' column and convert to float

df_bedrooms['no._of_bedrooms'] = df_bedrooms['no._of_bedrooms'].str.extract(r'(\d+)').astype(float)
df_bedrooms['no._of_bedrooms']

0        1.0
1        3.0
2        3.0
3        1.0
4        1.0
        ... 
22961    2.0
22962    1.0
22963    1.0
22964    1.0
22965    1.0
Name: no._of_bedrooms, Length: 22966, dtype: float64

In [301]:
# create rating column in original dateframe

df['no._of_bedrooms'] = df_bedrooms['no._of_bedrooms']


The `bedrooms` column has been cleaned and added to the original DataFrame.

### Working on beds column

In [302]:
# create beds dataframe

df_beds = pd.DataFrame()

In [303]:
# use bedroom and beds columns

df_bed = df_prop_inf[['no._of_bedrooms', 'no._of_beds']].copy()
df_bed.head()

,no._of_bedrooms,no._of_beds
0,1 bedroom,1 bed
1,3 bedrooms,4 beds
2,5 beds,2 baths
3,1 bedroom,2 beds
4,1 bed,1 bath


In [304]:
pattern = r'\b\d+\s+(?!bedrooms?\b)(?:beds?)\b'

# define a function to update the 'beds' column based on the pattern
def update_column(row):
    bedrooms_value = row['no._of_bedrooms']
    if bedrooms_value is None:
        return row['no._of_beds']
    elif re.findall(pattern, bedrooms_value):
        return bedrooms_value
    else:
        return row['no._of_beds']

# apply the update_column function to each row of the DataFrame 'df_bed'
df_bed['no._of_beds'] = df_bed.apply(update_column, axis=1)


In [305]:
total_bedroom_occurrences = df_bed['no._of_beds'].str.contains('no._of_beds').sum()
print("Total occurrences of 'bedroom' in 'beds' column:", total_bedroom_occurrences)

Total occurrences of 'bedroom' in 'beds' column: 0


In [306]:
df_bed.columns

Index(['no._of_bedrooms', 'no._of_beds'], dtype='object')

In [307]:
# extract integers from the 'beds' column and convert to float

df_bed['no._of_beds'] = df_bed['no._of_beds'].str.extract(r'(\d+)').astype(float)
df_bed['no._of_beds']

0         1.0
1         4.0
2         5.0
3         2.0
4         1.0
         ... 
22961     2.0
22962     1.0
22963     1.0
22964    19.0
22965     1.0
Name: no._of_beds, Length: 22966, dtype: float64

In [308]:
#create beds column in original dataframe

df['no._of_beds'] = df_bed['no._of_beds']


The `beds` column has been cleaned and added to the original DataFrame.

### Work on baths column

In [309]:
# create empty dataframe
df_baths = pd.DataFrame()

In [310]:
#use beds and bath columns and check dataframe

df_baths = df_prop_inf[['no._of_beds', 'no._of_baths']].copy()
df_baths.head()

,no._of_beds,no._of_baths
0,1 bed,1 bath
1,4 beds,3 baths
2,2 baths,None
3,2 beds,1.5 baths
4,1 bath,None


In [311]:
# define a function to update the 'baths' column based on the string values

def assign_baths(row):
    beds_value = row['no._of_beds']
    if beds_value is not None and 'bath' in beds_value:
        return beds_value
    else:
        return row['no._of_baths']

# apply the custom function to each row of the DataFrame
df_baths['no._of_baths'] = df_baths.apply(assign_baths, axis=1)


In [312]:
# calculate the total occurrences of 'bath' and without 'bath' in the 'baths' column
total_baths_occurrences = df_baths['no._of_baths'].str.contains('bath').sum()
total_occurrences_without_bath = total_rows - total_baths_occurrences

# calculate the total number of rows in the DataFrame
total_rows = len(df_baths)

# calculate the percentage of occurrences of 'bath' out of all the rows
percentage_baths_occurrences = (total_baths_occurrences / total_rows) * 100

# print the results
print("Total occurrences of 'bath' in 'baths' column:", total_baths_occurrences)
print("Percentage of occurrences of 'bath' out of all the rows:", percentage_baths_occurrences)
print("Total occurrences without 'bath' in 'baths' column:", total_occurrences_without_bath)


Total occurrences of 'bath' in 'baths' column: 22781
Percentage of occurrences of 'bath' out of all the rows: 99.19446137768875
Total occurrences without 'bath' in 'baths' column: -2379


In [313]:
# extract integers from the 'beds' column and convert to float

df_baths['no._of_baths'] = df_baths['no._of_baths'].str.extract(r'(\d+)').astype(float)
df_baths['no._of_baths']

0         1.0
1         3.0
2         2.0
3         1.0
4         1.0
         ... 
22961     1.0
22962     1.0
22963     1.0
22964    15.0
22965     1.0
Name: no._of_baths, Length: 22966, dtype: float64

In [314]:
# create baths column in original data frame

df['no._of_baths'] = df_baths['no._of_baths']
df['no._of_baths'].head()

0    1.0
1    3.0
2    2.0
3    1.0
4    1.0
Name: no._of_baths, dtype: float64


The `baths` column has been cleaned and added to the original DataFrame.

### 1.2 Preprocessing: Missing values

In [315]:
# Count the number of null rows

null_counts = df.isnull().sum()
total_rows = len(df)
null_percentage = (null_counts / total_rows) * 100
null_info = pd.DataFrame({
    'Number of Null Values': null_counts,
    'Percentage of Null Values': null_percentage
})

null_info[null_info['Number of Null Values'] > 0].sort_values(by='Percentage of Null Values', ascending=False)

,Number of Null Values,Percentage of Null Values
neighbourhood_group,22966,100.000000
license,22891,99.673430
rating,9339,40.664461
last_review,5764,25.097971
reviews_per_month,5764,25.097971
price,1392,6.061134
no._of_bedrooms,968,4.214926
no._of_baths,284,1.236611
no._of_beds,153,0.666202
host_name,4,0.017417


### Work on Neighbourhood Group column

In [316]:
# drop column

df.drop(columns=['neighbourhood_group'], inplace=True)

### Work on License column

In [317]:
# drop column

df.drop(columns=['license'], inplace=True)

### Work on Price column

In [318]:
df['price'].value_counts()

price
1500.0     373
1200.0     311
2500.0     294
2000.0     286
3000.0     266
          ... 
7523.0       1
412.0        1
13236.0      1
28750.0      1
805.0        1
Name: count, Length: 4824, dtype: int64

In [319]:
# print the shape of the DataFrame before and after removing rows with null values in the price column

print("Before removing rows with null values in price column there are", df.shape[0], "rows")
df.dropna(subset=['price'], inplace=True)
print("After removing rows with null values in price column there are", df.shape[0], "rows")


Before removing rows with null values in price column there are 22966 rows
After removing rows with null values in price column there are 21574 rows


### Work on bedrooms, baths & beds columns

In [320]:
df.dropna(subset=['no._of_bedrooms', 'no._of_baths', 'no._of_beds', 'host_name'], inplace=True)

In [321]:
# Count the number of null rows

null_counts = df.isnull().sum()
total_rows = len(df)
null_percentage = (null_counts / total_rows) * 100
null_info = pd.DataFrame({
    'Number of Null Values': null_counts,
    'Percentage of Null Values': null_percentage
})

null_info[null_info['Number of Null Values'] > 0].sort_values(by='Percentage of Null Values', ascending=False)

,Number of Null Values,Percentage of Null Values
rating,8348,40.925581
last_review,5119,25.095598
reviews_per_month,5119,25.095598


In [322]:
df.shape

(20398, 22)

#### Missing values summary:

- The `neighbourhood_group` and `license` columns were removed completely
- Rows that had missing values in either the `price`, `no._of_bedrooms`, `no._of_baths` or `no._of_beds` columns were removed

### 1.3 Preprocessing: Duplicate values

In [323]:
# Count the number of duplicated rows
num_duplicates = df.duplicated().sum()
total_rows = len(df)
duplicates_percentage = (num_duplicates / total_rows) * 100
duplicates_info = pd.DataFrame({
    'Number of Duplicated Rows': [num_duplicates],
    'Percentage of Duplicated Rows': [duplicates_percentage]
})

duplicates_info


,Number of Duplicated Rows,Percentage of Duplicated Rows
0,0,0.0


#### Duplicate values summary:

- There are no duplicates within the dataset

### Work on host name

In [337]:
host_name_counts = df.groupby('host_name')['host_id'].nunique()
duplicate_host_names = host_name_counts[host_name_counts > 1]

print("Host names with multiple host IDs:")
print(duplicate_host_names.sort_values(ascending=False))

Host names with multiple host IDs:
host_name
Propr       327
Michelle     63
Michael      59
David        52
James        47
           ... 
Junaid        2
Karlien       2
Abe           2
Kath          2
Zuzana        2
Name: host_id, Length: 1269, dtype: int64
